In [1]:
import pandas as pd
from datetime import datetime
import re
import json
import random


In [2]:
data = pd.read_excel('multi_agent_interaction_testing.xlsx')

In [3]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client['tutor_agent']
usercollection = db['user']
roomcollection = db['room']

In [4]:
image_paths = [
    "/src/assets/Bob.JPG",
    "/src/assets/ALice.JPG",
    "/src/assets/Jim.JPG",
    "/src/assets/Jack.JPG",
    "/src/assets/David.JPG",
]


In [5]:

def extract_profile(text):
    # 定义起始和结束标记
    start_marker = "here is your profile:"
    end_marker = "Your task:"
    
    # 查找起始位置
    start_index = text.find(start_marker)
    if start_index == -1:
        raise ValueError("起始标记未找到")
    start_index += len(start_marker)
    
    # 查找结束位置
    end_index = text.find(end_marker, start_index)
    if end_index == -1:
        raise ValueError("结束标记未找到")
    
    # 提取JSON字符串并清理
    json_str = text[start_index:end_index].strip()
    
    # 处理JSON字符串中的特殊结构
    # 注意：原始JSON中的键值都是字符串类型，无需额外转换
    try:
        profile_data = json.loads(json_str)
        return profile_data
    except json.JSONDecodeError as e:
        # 尝试修复常见的JSON格式问题
        # 移除可能存在的尾随逗号
        json_str = json_str.replace(',\n}', '\n}')
        json_str = json_str.replace(',\n]', '\n]')
        try:
            return json.loads(json_str)
        except:
            raise ValueError(f"JSON解析失败: {str(e)}") from e

In [6]:
for index, row in data.iterrows():
    member_num = row["person_num"]
    current_time = datetime.now()
    room = {
        "roomId": row["room_id"],
        "roomName": row["topic"],
        "roomCustomName": "",
        "memberNum": row["person_num"],
        "chatTime": "",
        "assertiveness": "",
        "topic": row["topic"],
        "stage_id": "stage1",
        "start_i": 0,
        "student_errors": [],
        "roomMember": [],
        "history": [
            {
                "date": current_time.strftime("%Y/%m/%d"),
                "time": current_time.strftime("%H:%M:%S"),
                "userId": "U001",
                "userName": "Teacher",
                "userAvatar": "/src/assets/Agent.PNG",
                "received_information": "",
                "response": f'Welcome! Today\'s topic is "{row["topic"]}".The entire discussion process should try to follow the five stages of problem definition, exploration, integration, resolution and feedback. Please brainstorm and express your opinions!',
            }
        ],
    }
    room_insert_result = roomcollection.insert_one(room)
    room_member_list = []
    for i in range(0, member_num):
        # 匹配 "You are " 后、逗号前的内容（非贪婪模式）
        match = re.search(r"You are (.*?),", row[f"role_{i}"])
        if match:
            name = match.group(1).strip()  # 提取分组并去除空格
            userAvatar = random.choice(image_paths)
            user = {
                "userName": name,
                "userAvatar": userAvatar,
                "userPswd": "",
                "mode": "student",
                "cognitiveLevel": {
                    "Joy": [0],
                    "Trust": [0],
                    "Fear": [0],
                    "Surprise": [0],
                    "Anger": [0],
                    "Disgust": [0],
                    "Engagement": [0],
                },
                "tracingX": [0],
                "profile": extract_profile(row[f"role_{i}"]),
                "roomList": [
                    {
                        "roomId": str(room_insert_result.inserted_id),
                        "roomName": row["topic"],
                    }
                ],
            }
            user_insert_result = usercollection.insert_one(user)
            room_member_item = {
                "memberId": str(user_insert_result.inserted_id),
                "memberName": name,
                "memberAvatar": userAvatar,
            }
            room_update_result = roomcollection.update_one(
                {"_id": room_insert_result.inserted_id},
                {"$push": {"roomMember": room_member_item}},
            )
        else:
            print("未找到名字")